<a href="https://colab.research.google.com/github/zxnb01/StudentWorkflow/blob/main/StudentWorkflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Student Workflow for learning

Setup & Install

In [20]:
# Install required libraries
!apt-get update -qq
!apt-get install -y ffmpeg
!pip install -q faster-whisper openai reportlab


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Import Libraries

In [21]:
import os
import subprocess
from google.colab import files


Upload Lecture File

In [22]:
uploaded = files.upload()

lecture_file = list(uploaded.keys())[0]
print("Uploaded:", lecture_file)


Saving 16min_long_lecture.mp4 to 16min_long_lecture.mp4
Uploaded: 16min_long_lecture.mp4


Extract Audio

In [23]:
audio_file = "audio.wav"

subprocess.run([
    "ffmpeg",
    "-i", lecture_file,
    "-ac", "1",
    "-ar", "16000",
    audio_file,
    "-y"
])

print("Audio extracted")


Audio extracted


Transcription

In [24]:
from faster_whisper import WhisperModel

model = WhisperModel("base", device="cuda", compute_type="float16")

segments, info = model.transcribe(audio_file)

transcript = ""
for seg in segments:
    transcript += seg.text + " "

os.makedirs("outputs", exist_ok=True)
open("outputs/transcript.txt", "w").write(transcript)

print("Transcription complete")


Transcription complete


Study Notes Generation

In [25]:
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get("GEMINI_API_KEY")
if not api_key:
    raise ValueError("Gemini API key not found in Colab secrets")

genai.configure(api_key=api_key)


In [26]:
!pip install -q google-generativeai


In [27]:
model = genai.GenerativeModel("gemini-2.5-flash")
notes_prompt = f"""
You are an expert tutor.
Convert the following lecture transcript into:
- Structured study notes
- Bullet points
- Key definitions
- Exam-oriented highlights

Transcript:
{transcript}
"""

response = model.generate_content(notes_prompt)
notes = response.text

import os
os.makedirs("outputs", exist_ok=True)
open("outputs/notes.md", "w").write(notes)

print("Notes generated")


Notes generated


Flashcards Generation

In [28]:
flash_prompt = f"""
Generate flashcards in Question/Answer format from the lecture.
Return them as plain text.

Lecture:
{transcript}
"""

flashcards = model.generate_content(flash_prompt).text
open("outputs/flashcards.txt", "w").write(flashcards)


5772

Export to PDF

In [29]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas

pdf_path = "outputs/study_material.pdf"
c = canvas.Canvas(pdf_path, pagesize=A4)

text = c.beginText(40, 800)
text.setFont("Helvetica", 10)

for line in notes.split("\n"):
    text.textLine(line)

c.drawText(text)
c.save()

print("PDF created:", pdf_path)


PDF created: outputs/study_material.pdf


Download Outputs

In [30]:
files.download("outputs/study_material.pdf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>